# Imports and styles

In [ ]:
import simpy
from lib.core import NetworkTap, Switch, PacketSource, PacketSink, SwitchPort, PacketFork
import numpy as np
from functools import partial
import matplotlib.pyplot as plt
import matplotlib.image as image
from lib.params import colors, rc_params
from IPython.display import display, HTML

# Define a style for the dashboard
dashboard_style = """
<style>
  .dashboard {
      display: grid;
      grid-template-columns: repeat(2, 1fr);
      gap: 20px;
      margin: 20px 0;
  }
  .card {
      background-color: #f9f9f9;
      border-radius: 8px;
      padding: 15px;
      box-shadow: 0 2px 4px rgba(0, 0, 0, 0.1);
  }
  .card h3 {
      margin-top: 0;
      color: #333;
  }
  .card p {
      margin: 5px 0;
      color: #555;
  }
</style>
"""

%load_ext autoreload
%autoreload 2

rng = np.random.default_rng()

# Utility functions

In [ ]:
def add_logo(f):
    im = image.imread("logo.png")
    # put a new axes where you want the image to appear
    # (x, y, width, height)
    imax = fig.add_axes([.83, .9, 0.1, 0.1])
    # remove ticks & the box from imax 
    imax.set_axis_off()
    # print the logo with aspect="equal" to avoid distorting the logo
    imax.imshow(im, aspect="equal")
    return

# Credits
The original idea and implementation for this exercise comes from the website of Greg Bernstein that can be viewed here: https://www.grotto-networking.com/DiscreteEventPython.html

# Uniform distribution

https://numpy.org/doc/stable/reference/random/generated/numpy.random.Generator.random.html#numpy.random.Generator.random

In [ ]:
SIZE = 10000

In [ ]:
x = rng.random(size=SIZE)
x, x.min(),x.mean(),x.max()

In [ ]:
plt.rcParams.update(rc_params)
fig, ax = plt.subplots(figsize=(10,6))

add_logo(fig)

ax.hist(x, color=colors[0], rwidth=0.5, bins=50)
ax.set_xlabel("Value [-]")
ax.set_ylabel("Frequency [-]")
ax.set_title("Histogram of Uniformly Distributed Var.")
plt.show()

# Exponential distribution

https://numpy.org/doc/stable/reference/random/generated/numpy.random.Generator.exponential.html

In [ ]:
x = rng.exponential(scale=1, size=SIZE)
x, x.min(),x.mean(),x.max()

In [ ]:
plt.rcParams.update(rc_params)
fig, ax = plt.subplots(figsize=(10,6))

add_logo(fig)

ax.hist(x, color=colors[0], rwidth=0.5, bins=50)
ax.set_xlabel("Value [-]")
ax.set_ylabel("Frequency [-]")
ax.set_title("Histogram of Exp. Distributed Var.")
plt.show()

# Normal distribution

https://numpy.org/doc/stable/reference/random/generated/numpy.random.Generator.normal.html

In [ ]:
x = rng.normal(loc=0, scale=1, size=SIZE)
x, x.min(),x.mean(),x.max()

In [ ]:
plt.rcParams.update(rc_params)
fig, ax = plt.subplots(figsize=(10,6))

add_logo(fig)

ax.hist(x, color=colors[0], rwidth=0.5, bins=50)
ax.set_xlabel("Value [-]")
ax.set_ylabel("Frequency [-]")
ax.set_title("Histogram of Normally Distributed Var.")
plt.show()

# Simulation Source - Sink

In [ ]:
# create Simpy environment
env = simpy.Environment()

# create simple sink for packets
sink = PacketSink(env, "sink", debug=True)

# create a packet source
source = PacketSource(
    env,
    "source01",
    packet_interval=1, # interval between individual packets in simulation units
    packet_size=10 # packet size in Bytes
)

# link them together
source.destination = sink

# run the simulation
simulation_time = 20
env.run(until=simulation_time)

## Task
* Modify the simulation code above to have 2 packet sources with varying sizes and interarrival times.
* Using the built-in partial function, experiment with the use of statistical distributions for packet sizes and interarrivals.

# Simulation Source - Switch - Sink

In [ ]:
env = simpy.Environment()

sink = PacketSink(env, "sink", debug=True)

switch = Switch(
    env,
    "switch01",
    num_ports=4, # how many ports does the switch have. IMPORTANT: port is a bi-partisan connection so it actually represents 2 physical ports on a real-world switch
    port_capacity=100, # queue lenght in Bytes
    port_transmission_rate=1000 # bits/s
) 

source = PacketSource(env, "source01", packet_interval=partial(rng.normal, 2,0.5), packet_size=partial(rng.exponential, 50))


source.destination = switch.ports[0]
switch.ports[0].destination = sink

simulation_time = 200
env.run(until=simulation_time)

## Task
- How long were packets in the system? 

# Simulation Source - Switch - Sink with Network Tap

In [ ]:
env = simpy.Environment()

sink = PacketSink(env, "sink", debug=True)

switch = Switch(
    env,
    "switch01",
    num_ports=4, # how many ports does the switch have. IMPORTANT: port is a bi-partisan connection so it actually represents 2 physical ports on a real-world switch
    port_capacity=100, # queue lenght in Bytes
    port_transmission_rate=1000 # bits/s
)
tap = NetworkTap(
    env,
    port=switch.ports[0] # port to be monitored
)

source = PacketSource(env, "source01", packet_interval=partial(rng.normal, 2,0.5), packet_size=partial(rng.exponential, 50))


source.destination = switch.ports[0]
switch.ports[0].destination = sink

simulation_time = 200
env.run(until=simulation_time)

In [ ]:
tap

# M/M/1 System

### Simulation

In [ ]:
adist = partial(rng.exponential, 2) 
sdist = partial(rng.exponential, 100) # mean size 100 bytes


env = simpy.Environment()  # Create the SimPy environment
# Create the packet generators and sink
ps = PacketSink(env, sink_id="Sink1", debug=False)
pg = PacketSource(env, "Generator1", packet_interval=adist, packet_size=sdist, debug=False)

switch = Switch(
    env,
    "switch01",
    num_ports=1, # how many ports does the switch have. IMPORTANT: port is a bi-partisan connection so it actually represents 2 physical ports on a real-world switch
    port_capacity=10000, # queue lenght in Bytes
    port_transmission_rate=1000 # bits/s
)
tap = NetworkTap(
    env,
    port=switch.ports[0] # port to be monitored
)

pg.destination = switch.ports[0]
switch.ports[0].destination = ps

# Run it
env.run(until=8000)

### Display Dashboard and Chart

In [ ]:

# Display the style
display(HTML(dashboard_style))

# Create the dashboard layout
dashboard_html = f"""
<div class="dashboard">
  <div class="card">
      <h3>Last 10 Waits</h3>
      <p>{', '.join([f'{x:.3f}' for x in ps.delays[-10:]])}</p>
  </div>
  <div class="card">
      <h3>Last 10 packet counts on Tap</h3>
      <p>{', '.join(f'{x}' for x in tap.packet_count[-10:])}</p>
  </div> 
  <div class="card">
      <h3>Last 10 byte counts on Tap</h3>
      <p>{', '.join(f'{x}' for x in tap.byte_count[-10:])}</p>
  </div>
  <div class="card">
      <h3>Last 10 Sink Arrival Times</h3>
      <p>{', '.join([f'{x:.3f}' for x in ps.arrivals[-10:]])}</p>
  </div>
  <div class="card">
      <h3>Average Wait</h3>
      <p>{sum(ps.delays) / len(ps.delays):.3f}</p>
  </div>
  <div class="card">
      <h3>Processed and Dropped</h3>
      <p>Processed: {switch.ports[0].cum_packet_count}, Dropped: {switch.ports[0].cum_drop_count}</p>
  </div>
  <div class="card">
      <h3>Loss Rate</h3>
      <p>{float(switch.ports[0].cum_drop_count) / switch.ports[0].cum_packet_count:.3f}</p>
  </div>
  <div class="card">
      <h3>Average System Occupancy</h3>
      <p>{float(sum(tap.packet_count)) / len(tap.packet_count):.3f}</p>
  </div>
</div>
"""

# Display the dashboard
display(HTML(dashboard_html))


plt.rcParams.update(rc_params)
fig, ax = plt.subplots(figsize=(10,6))
add_logo(fig)
ax.hist(ps.interarrivals, bins=100, color=colors[0], rwidth=0.8)
ax.set_xlabel("Time [STU]")
ax.set_ylabel("Frequency of Occurence [-]")
ax.set_title("Histogram of Inter-arrivals")
plt.show()

# More complex systems of queues

### Simulation

In [ ]:
mean_pkt_size = 100.0  # in bytes

adist1 = partial(rng.exponential, 0.5)
adist2 = partial(rng.exponential, 2)
adist3 = partial(rng.exponential, 10/6)

sdist = partial(rng.exponential, mean_pkt_size)

samp_dist = partial(rng.exponential, 2)


port_rate = 2.2*8*mean_pkt_size  # want a rate of 2.2 packets per second

# Create the SimPy environment. This is the thing that runs the simulation.
env = simpy.Environment()


ps1 = PacketSink(env, sink_id="Sink1", debug=False)
ps2 = PacketSink(env, sink_id="Sink2", debug=False)
pg1 = PacketSource(env, "GEN1", packet_interval=adist1, packet_size=sdist)
pg2 = PacketSource(env, "GEN2", packet_interval=adist2, packet_size=sdist)
pg3 = PacketSource(env, "GEN3", packet_interval=adist3, packet_size=sdist)
fork1 = PacketFork(env, [0.75, 0.25])
fork2 = PacketFork(env, [0.65, 0.35])


switch = Switch(
    env,
    "switch01",
    num_ports=4, # how many ports does the switch have. IMPORTANT: port is a bi-partisan connection so it actually represents 2 physical ports on a real-world switch
    port_capacity=10000, # queue lenght in Bytes
    port_transmission_rate=port_rate # bits/s
)
tap = NetworkTap(
    env,
    port=switch.ports[0] # port to be monitored
)

pg.destination = switch.ports[0]
switch.ports[0].destination = ps

# Wire packet generators, switch ports, and sinks together
pg1.destination = switch.ports[0]
switch.ports[0].destination = fork1
fork1.destinations[0] = switch.ports[1]
switch.ports[1].destination = fork2
fork2.destinations[0] = switch.ports[2]
fork2.destinations[1] = switch.ports[3]
pg3.destination = switch.ports[2]
pg2.destination = switch.ports[3]
switch.ports[2].destination = ps1
switch.ports[3].destination = ps2
# Run it
env.run(until=4000)

### Dashboard and Chart

In [ ]:

# Display the style
display(HTML(dashboard_style))

# Create the dashboard layout
dashboard_html = f"""
<div class="dashboard">
  <div class="card">
      <h3>Last 10 Waits on PS2</h3>
      <p>{', '.join([f'{x:.3f}' for x in ps2.delays[-10:]])}</p>
  </div>
  <div class="card">
      <h3>Average Port 0 Occupancy</h3>
      <p>{float(sum(tap.packet_count)) / len(tap.packet_count):.3f}</p>
  </div>
</div>
"""

# Display the dashboard
display(HTML(dashboard_html))


plt.rcParams.update(rc_params)
fig, ax = plt.subplots(figsize=(10,6))
add_logo(fig)
ax.hist(ps1.interarrivals, bins=100, color=colors[0], rwidth=0.8, alpha=0.5)
ax.hist(ps2.interarrivals, bins=100, color=colors[1], rwidth=0.8, alpha=0.5)
ax.set_xlabel("Time [STU]")
ax.set_ylabel("Frequency of Occurence [-]")
ax.set_title("Histogram of Inter-arrivals")
plt.show()